###`Product Data POC  on Structured Streaming`

`This notebook we are using for to do POC on structured Streaming `:
* Reading multiple csv files using structured streaming 
* Steaming data load of weather data using delta table


###Details
| Details | Information
| - | -  
| Notebook Created By | Gvnreddy  
| Object Name | product data analysis
| File Type | delimited file
| Target Location | Databricks Delta Table 

###History
|Date | Developed By | comments
|----|-----|----
|01/12/2023|Gvnreddy| Initial Version

#### calling `common` notebook for metadata and other variables

In [0]:
%run ./config_streaming

path,name,size,modificationTime
dbfs:/FileStore/products/,products/,0,0
dbfs:/FileStore/shared_uploads/,shared_uploads/,0,0
dbfs:/FileStore/tables/,tables/,0,0


weather streamings files copied to : dbfs:/FileStore/streaming


Out[4]: True

path,name,size,modificationTime
dbfs:/FileStore/products/,products/,0,0
dbfs:/FileStore/shared_uploads/,shared_uploads/,0,0
dbfs:/FileStore/streaming/,streaming/,0,0
dbfs:/FileStore/tables/,tables/,0,0


In [0]:
%fs head dbfs:/FileStore/streaming/ws01.txt

date	actual_mean_temp	actual_min_temp	actual_max_temp	average_min_temp	average_max_temp	record_min_temp	record_max_temp	record_min_temp_year	record_max_temp_year	actual_precipitation	average_precipitation	record_precipitation
01-07-2014	81	70	91	67	89	56	104	1919	2012	0	0.1	5.91
02-07-2014	85	74	95	68	89	56	101	2008	1931	0	0.1	1.53
03-07-2014	82	71	93	68	89	56	99	2010	1931	0.14	0.11	2.5
04-07-2014	75	64	86	68	89	55	99	1933	1955	0	0.1	2.63
05-07-2014	72	60	84	68	89	57	100	1967	1954	0	0.1	1.65
06-07-2014	74	61	87	68	89	57	99	1964	1948	0	0.1	1.95
07-07-2014	79	67	91	68	89	55	100	1972	1954	0	0.11	2.37
08-07-2014	83	72	94	68	89	58	101	1892	2010	0	0.11	1.87
09-07-2014	80	71	89	68	89	57	101	1891	1986	0.15	0.12	3.71
10-07-2014	78	71	85	68	89	53	101	1961	1926	0	0.11	2.45
11-07-2014	78	68	87	68	89	55	100	1961	1986	0	0.12	3.1
12-07-2014	79	67	91	68	89	59	102	1953	1930	0	0.11	1.95
13-07-2014	81	69	92	68	89	56	100	1918	1986	0	0.12	2.09
14-07-2014	85	74	96	68	89	59	102	2007	1954	0	0.12	2.15
15-07-2014	81	67	94	68	89	58	100	2001	1893	0.87	0.12	4.61
16-07-2014	76	66	85	68	89	58	100	2001	1887	0	0.12	1.47
17-07-2014	74	63	84	68	89	62	100	1886	1986	0	0.12	4.14
18-07-2014	77	70	83	68	89	62	102	1896	1887	0	0.12	1.82
19-07-2014	70	66	74	68	89	60	103	2009	1986	0.17	0.12	2.3
20-07-2014	74	65	83	68	89	60	101	1910	1986	0	0.12	1.25
21-07-2014	74	70	77	68	89	57	103	1890	1986	1.3	0.12	3.83
22-07-2014	79	72	85	68	89	58	103	2001	1926	0	0.12	2.56
23-07-2014	80	69	90	68	89	56	101	2007	1987	0	0.13	6.88
24-07-2014	80	71	89	68	89	59	100	1947	1952	0.06	0.12	2.83
25-07-2014	77	68	86	68	89	62	101	1947	2010	0	0.13	2.34
26-07-2014	78	66	89	68	89	60	100	1904	2005	0	0.13	2.84
27-07-2014	80	70	90	68	89	57	103	1920	1940	0.18	0.13	2.46
28-07-2014	79	68	90	68	89	54	102	1962	1952	0.06	0.13	2.98
29-07-2014	73	62	83	68	89	59	103	1920	1952	0	0.14	2.04
30-07-2014	74	63	85	68	89	58	101	1914	2011	0	0.14	4.24
31-07-2014	72	65	78	68	89	56	100	1914	1999	1.45	0.14	1.45
01-08-2014	68	65	70	68	89	58	98	1966	1957	2.39	0.14	2.39
02-08-2014	69	65	72	68	89	59	98	1895	1942	0	0.14	2
03-08-2014	77	67	86	68	89	60	98	1998	2011	0	0.15	4.51
04-08-2014	78	69	87	68	89	58	98	1912	1935	0	0.15	2.59
05-08-2014	77	65	88	68	89	55	100	1912	1954	0	0.15	3.01
06-08-2014	78	68	88	68	88	57	102	1912	1956	0.17	0.14	3.19
07-08-2014	78	67	88	68	88	50	100	2004	1963	0	0.13	1.53
08-08-2014	77	73	81	68	88	55	102	2004	2007	0	0.13	1.37
09-08-2014	76	68	84	68	88	56	104	2002	2007	0	0.13	1.26
10-08-2014	70	67	73	68	88	56	104	1879	2007	0.01	0.13	2.26
11-08-2014	78	71	85	68	88	58	98	1879	1956	0.58	0.13	3.04
12-08-2014	81	72	89	68	88	57	100	1890	1925	0.31	0.13	2.25
13-08-2014	75	65	84	68	88	57	102	1979	1956	0	0.14	1.27
14-08-2014	71	59	83	68	88	57	98	1967	1925	0	0.14	2
15-08-2014	73	60	86	68	88	55	99	1964	2007	0	0.14	3.17
16-08-2014	77	68	86	68	88	57	102	1924	1954	0	0.14	2.51
17-08-2014	78	67	89	67	88	55	97	1979	1911	0	0.14	2.92
18-08-2014	81	71	90	67	88	59	101	1976	1988	0	0.13	1.76
19-08-2014	80	70	90	67	87	59	100	1981	1988	0	0.14	1.96
20-08-2014	79	67	91	67	87	56	103	1918	1925	0	0.14	1.84
21-08-2014	80	67	93	67	87	56	103	1998	1983	0	0.13	2.22
22-08-2014	83	70	95	67	87	57	101	1886	1983	0	0.15	2.81
23-08-2014	82	73	91	67	87	53	103	1930	1983	0.08	0.13	2.74
24-08-2014	74	66	81	67	87	55	97	1888	1968	0.14	0.14	2.35
25-08-2014	72	60	83	66	87	56	98	1952	1943	0	0.14	3.16
26-08-2014	72	60	84	66	87	55	100	1942	1954	0	0.14	5.36
27-08-2014	72	57	87	66	86	55	101	1952	1954	0	0.13	4.39
28-08-2014	77	61	92	66	86	55	98	1905	1924	0	0.13	4.64
29-08-2014	79	69	88	66	86	54	100	1986	1948	0	0.13	1.42
30-08-2014	79	68	90	66	86	53	101	1965	1932	0	0.12	2.54
31-08-2014	83	74	91	65	86	53	99	1946	1957	0.06	0.12	3.4
01-09-2014	82	71	93	65	85	51	99	1999	1957	0	0.11	2.93
02-09-2014	82	69	94	65	85	53	99	1967	1932	0	0.12	1.2
03-09-2014	81	68	93	65	85	52	98	1967	1925	0.63	0.11	2.67
04-09-2014	77	69	85	64	85	54	100	1952	1925	0.82	0.11	1.15
05-09-2014	79	69	89	64	84	54	101	1967	1954	0.01	0.11	2.

##### Creating pyspark schema for reading csv files.

#### Creating streaming dataframe using `maxFilesPerTrigger 1 option`

In [0]:
from pyspark.sql.functions import *

# Similar to definition of staticInputDF above, just using `readStream` instead of `read`
streamingInputDF = spark.readStream.schema(streamSchema).option("maxFilesPerTrigger", 1).csv(streaming_path,header=True,sep="\t")

##### Validating is above created dataframe is Streaming dataframe or not

In [0]:
streamingInputDF.isStreaming

Out[6]: True

In [0]:
if streamingInputDF.isStreaming==True:
  print(' streamingInputDF is Streaming Dataframe')

 streamingInputDF is Streaming Dataframe


##### Perform the following transformations
###### a.	Drop the following columns :  record_min_temp_year, record_max_temp_year
###### b.	Filter out records with actual_precipitation = 0
###### c.	Create a new column deviation_from_avg = abs(actual_precipitation – average_precipitation)


In [0]:
# extracting year from a date column date_format(column,'yyyy')
# extracting month from a date column date_format(colmn,'MM')

In [0]:
%sql
select date_format(current_date(),'yyyy') as year,date_format(current_date(),'MM') as month,date_format(current_date(),'dd') as day

year,month,day
2024,02,01


In [0]:
from pyspark.sql.functions import abs,col,date_format
finalStreamingDF = streamingInputDF.drop("record_min_temp_year","record_max_temp_year").filter("actual_precipitation ==0").withColumn("deviation_from_avg",abs(col("actual_precipitation")-col("average_precipitation")))
finalStreamingDF = finalStreamingDF.withColumn("date",to_date("date","dd-MM-yyyy")).withColumn("YEAR",date_format(col("date"),'yyyy')).withColumn("MONTH",date_format(col("date"),'MM'))

In [0]:
# creating partition table using year and month columns
writedf = finalStreamingDF.writeStream.format("delta").partitionBy("YEAR","MONTH").outputMode("append").option("checkpointLocation", "/tmp/_checkpoints/streaming").start("/tmp/streaming")

##### Stop Streaming using `stop()` method

In [0]:
import time
time.sleep(1*60) #this will stop the program for 1 minutes
writedf.stop()

In [0]:
%fs ls /tmp/streaming/

path,name,size,modificationTime
dbfs:/tmp/streaming/YEAR=2014/,YEAR=2014/,0,0
dbfs:/tmp/streaming/YEAR=2015/,YEAR=2015/,0,0
dbfs:/tmp/streaming/_delta_log/,_delta_log/,0,0


In [0]:
%sql
select * from delta.`/tmp/streaming`  where year=2014 and month=10

date,actual_mean_temp,actual_min_temp,actual_max_temp,average_min_temp,average_max_temp,record_min_temp,record_max_temp,actual_precipitation,average_precipitation,record_precipitation,deviation_from_avg,YEAR,MONTH
2014-10-01,76,70,82,66,84,48,94,0.0,0.21,5.14,0.21,2014,10
2014-10-02,78,70,85,65,84,43,93,0.0,0.2,3.57,0.2,2014,10
2014-10-05,60,47,73,64,83,46,94,0.0,0.18,4.51,0.18,2014,10
2014-10-06,64,50,78,64,83,49,95,0.0,0.17,3.48,0.17,2014,10
2014-10-07,72,60,84,64,82,47,95,0.0,0.18,6.14,0.18,2014,10
2014-10-08,76,63,89,63,82,44,95,0.0,0.17,3.91,0.17,2014,10
2014-10-09,76,64,88,63,82,46,94,0.0,0.17,1.8,0.17,2014,10
2014-10-10,76,65,87,63,82,46,93,0.0,0.16,3.56,0.16,2014,10
2014-10-11,75,62,88,62,82,45,91,0.0,0.16,4.3,0.16,2014,10
2014-10-12,74,62,86,62,81,46,92,0.0,0.15,2.31,0.15,2014,10


In [0]:
df = spark.read.format("delta").load("/tmp/streaming")

In [0]:
%fs ls /tmp/streaming/YEAR=2014/MONTH=10/

path,name,size,modificationTime
dbfs:/tmp/streaming/YEAR=2014/MONTH=10/part-00000-0dd7cab8-903f-42c1-904d-031e1892aa83.c000.snappy.parquet,part-00000-0dd7cab8-903f-42c1-904d-031e1892aa83.c000.snappy.parquet,4553,1706785352000
dbfs:/tmp/streaming/YEAR=2014/MONTH=10/part-00000-11389382-43da-4227-955e-e98ba385beed.c000.snappy.parquet,part-00000-11389382-43da-4227-955e-e98ba385beed.c000.snappy.parquet,4339,1706785390000
dbfs:/tmp/streaming/YEAR=2014/MONTH=10/part-00000-21d7ae93-861a-4b71-a4ff-e0f998c9eff5.c000.snappy.parquet,part-00000-21d7ae93-861a-4b71-a4ff-e0f998c9eff5.c000.snappy.parquet,4641,1706785384000
dbfs:/tmp/streaming/YEAR=2014/MONTH=10/part-00000-25a15e9a-430f-4136-bbed-4ac02c4e3df0.c000.snappy.parquet,part-00000-25a15e9a-430f-4136-bbed-4ac02c4e3df0.c000.snappy.parquet,4624,1706785346000
dbfs:/tmp/streaming/YEAR=2014/MONTH=10/part-00000-5b5bf96a-e325-46cf-aeb7-b6243551f410.c000.snappy.parquet,part-00000-5b5bf96a-e325-46cf-aeb7-b6243551f410.c000.snappy.parquet,4523,1706785368000
dbfs:/tmp/streaming/YEAR=2014/MONTH=10/part-00000-682f92fe-e008-43df-9dcb-ee8c18d23215.c000.snappy.parquet,part-00000-682f92fe-e008-43df-9dcb-ee8c18d23215.c000.snappy.parquet,4339,1706785395000
dbfs:/tmp/streaming/YEAR=2014/MONTH=10/part-00000-77b01ac5-ce48-4072-9044-8729f9343cfe.c000.snappy.parquet,part-00000-77b01ac5-ce48-4072-9044-8729f9343cfe.c000.snappy.parquet,4849,1706785363000
dbfs:/tmp/streaming/YEAR=2014/MONTH=10/part-00000-9b0421df-21b2-461a-b8f6-15fc00de8808.c000.snappy.parquet,part-00000-9b0421df-21b2-461a-b8f6-15fc00de8808.c000.snappy.parquet,4442,1706785358000
dbfs:/tmp/streaming/YEAR=2014/MONTH=10/part-00000-c770cf50-4f4a-4aae-9aeb-ab0c1b2a59b9.c000.snappy.parquet,part-00000-c770cf50-4f4a-4aae-9aeb-ab0c1b2a59b9.c000.snappy.parquet,4567,1706785374000
dbfs:/tmp/streaming/YEAR=2014/MONTH=10/part-00000-d36d5594-1cd4-40ac-87a4-c275ddb0e65f.c000.snappy.parquet,part-00000-d36d5594-1cd4-40ac-87a4-c275ddb0e65f.c000.snappy.parquet,4658,1706785339000
